## Homework №2, done by Toktonyazov Ismail

## Task#1

In [98]:
import pandas as pd
import numpy as np
import numpy.linalg as npla
import statsmodels.api as sm
import scipy.stats as st
import scipy.optimize as spopt

In [4]:
data = sm.datasets.get_rdataset("Guerry", "HistData").data

X=data[['Literacy','Donations','Infants', 'Wealth', 'Commerce']]
X=sm.add_constant(X)

Y=data['Lottery']

In [5]:
smfOLS = sm.regression.linear_model.OLS.from_formula

In [6]:
def assignment_1_true(formula, data):
    
    fit = smfOLS(formula, data = data).fit()
    
    return fit.conf_int()

In [7]:
assignment_1_true('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', data)

,0,1
Intercept,-6.027115,41.046645
Literacy,-0.479342,0.179528
Donations,-0.000675,0.000954
Infants,-0.000005,0.001164
Wealth,0.096173,0.516922
Commerce,-0.100867,0.405366


In [8]:
def assignment_1(formula, data):
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    X = data[xcolumns].values
    X = sm.add_constant(X)
    
    # My code is here:
    beta = npla.solve(X.T@X, X.T@Y)
    e = Y - X@beta
    cov_matrix = (e.T@e)/(X.shape[0] - X.shape[1]) * npla.inv(X.T@X)
    std_errors = np.sqrt(np.diagonal(cov_matrix))
    t_score = st.t.ppf((1 + 0.95) / 2, X.shape[0] - X.shape[1])
    
    results = [[beta[i] - t_score * std_errors[i], beta[i] + t_score * std_errors[i]] for i in range(6)]
    
    df = pd.DataFrame(results, columns = [0,1])
    df.index = np.array(['Intercept'] + xcolumns)
    
    return df

In [9]:
assignment_1('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', data)

,0,1
Intercept,-6.027115,41.046645
Literacy,-0.479342,0.179528
Donations,-0.000675,0.000954
Infants,-0.000005,0.001164
Wealth,0.096173,0.516922
Commerce,-0.100867,0.405366


## Task#2

In [68]:
def loss_elastic_net(X, Y, beta, lamda, mu):
    e = Y - X@beta
    return e@e.T + lamda * beta[1:]@beta[1:].T + mu * abs(beta[1:]).sum()

In [69]:
def assignment_2(formula, data, mu, lamda):
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    X = data[xcolumns].values
    X = sm.add_constant(X)
    
    # My code is here:
    covs = 6
    bounds = [(None, None)] * covs
    results = spopt.shgo(lambda beta: loss_elastic_net(X, Y, beta, lamda, mu), bounds).x
    
    df = pd.DataFrame(results, columns = [0])
    df.index = np.array(['Intercept'] + xcolumns)
    
    return df

In [70]:
assignment_2('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', data, 1, 1)

,0
Intercept,17.509735
Literacy,-0.149888
Donations,0.000140
Infants,0.000579
Wealth,0.306536
Commerce,0.152254


## Task#3

In [94]:
from sklearn.model_selection import KFold

def SSR(X, y, lamda, mu, folds):
    avg = 0
    covs = 6
    bounds = [(None, None)] * covs
    kf = KFold(n_splits=folds)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        beta = spopt.shgo(lambda beta: loss_elastic_net(X_train, y_train, beta, lamda, mu), bounds).x
        errors = X_test@beta - y_test
        SSR = errors@errors.T
        avg = avg + SSR
    return avg/folds

In [95]:
def assignment_3(formula, data, folds=5):
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    X = data[xcolumns].values
    X = sm.add_constant(X)
    
    # My code is here:
    covs = 6
    bounds = [(None, None)] * covs
    SSR_results = []
    for lamda in np.linspace(0, 0.5, 10):
        for mu in np.linspace(0, 0.5, 10):
            SSR_results.append((SSR(X, Y, lamda, mu, folds), lamda, mu))
    SSR_results.sort(key=lambda x: x[0])
    optimal_lamda = SSR_results[0][1]
    optimal_mu = SSR_results[0][2]
    results = spopt.shgo(lambda beta: loss_elastic_net(X, Y, beta, optimal_lamda, optimal_mu), bounds).x
    
    df = pd.DataFrame(results, columns = [0])
    df.index = np.array(['Intercept'] + xcolumns)
    
    return df

In [96]:
assignment_3('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', data, folds=5)

,0
Intercept,17.510310
Literacy,-0.149902
Donations,0.000140
Infants,0.000579
Wealth,0.306544
Commerce,0.152254


## Task#4

The covariance matrix of the Ridge estimator is:<br>
<p style="text-align: center;">$Var[\hat{\beta}_{\lambda}|X]=\sigma^{2}(X^{\top}X+\lambda I)^{-1}X^{\top}X[(X^{\top}X+\lambda I)^{-1}]^{\top}$</p><br>
Hence, the standard errors of the beta coefficients are the elements on the main diagonal of the covariance matrix from which the square root is taken.<br>
<b>Proof (covariance matrix calculation):<br></b>
Remember that the OLS estimator $\hat{\beta}$ has conditional variance:<br>
<p style="text-align: center;">$Var[\hat{\beta}|X]=\sigma^{2}(X^{\top}X)^{-1}$</p><br>
We can write the ridge estimator as a function of the OLS estimator:<br>
<p style="text-align: center;">$\hat{\beta}_{\lambda}=(X^{\top}X+\lambda I)^{-1}X^{\top}y=$<br></p>
<p style="text-align: center;"><br>$=(X^{\top}X+\lambda I)^{-1}X^{\top}X(X^{\top}X)^{-1}X^{\top}y=$<br></p>
<p style="text-align: center;"><br>$=(X^{\top}X+\lambda I)^{-1}X^{\top}X\hat{\beta}$<br></p>
Therefore:<br>
We will use the following fact: $Var[AY]=AVar(Y)A^{\top}$
<p style="text-align: center;">$Var[\hat{\beta}_{\lambda}|X]=$<br></p>
<p style="text-align: center;">$=(X^{\top}X+\lambda I)^{-1}X^{\top}XVar[\hat{\beta}|X][(X^{\top}X+\lambda I)^{-1}X^{\top}X]^{\top}=$<br></p>
<p style="text-align: center;">$=(X^{\top}X+\lambda I)^{-1}X^{\top}XVar[\hat{\beta}|X]X^{\top}X[(X^{\top}X+\lambda I)^{-1}]^{\top}=$<br></p>
<p style="text-align: center;">$=(X^{\top}X+\lambda I)^{-1}X^{\top}X\sigma^{2}(X^{\top}X)^{-1}X^{\top}X[(X^{\top}X+\lambda I)^{-1}]^{\top}=$<br></p>
<p style="text-align: center;">$=\sigma^{2}(X^{\top}X+\lambda I)^{-1}X^{\top}X[(X^{\top}X+\lambda I)^{-1}]^{\top}$</p>

In [101]:
def loss_ridge(X, Y, beta, lamda):
    e = Y - X@beta
    return e@e.T + lamda * beta[1:]@beta[1:].T

In [102]:
def SSR_ridge(X, y, lamda, folds):
    avg = 0
    covs = 6
    bounds = [(None, None)] * covs
    kf = KFold(n_splits=folds)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        beta = spopt.shgo(lambda beta: loss_ridge(X_train, y_train, beta, lamda), bounds).x
        errors = X_test@beta - y_test
        SSR = errors@errors.T
        avg = avg + SSR
    return avg/folds

In [120]:
def assignment_4(formula, data, folds=5):
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    X = data[xcolumns].values
    X = sm.add_constant(X)
    
    # My code is here:
    covs = 6
    bounds = [(None, None)] * covs
    SSR_results = []
    for lamda in np.linspace(0, 0.5, 10):
        SSR_results.append((SSR_ridge(X, Y, lamda, folds), lamda))
    SSR_results.sort(key=lambda x: x[0])
    optimal_lamda = SSR_results[0][1]
    beta = spopt.shgo(lambda beta: loss_ridge(X, Y, beta, optimal_lamda), bounds).x
    e = Y - X@beta
    delta_2 = e.T@e/(X.shape[0] - X.shape[1])
    cov_matrix = delta_2*npla.inv(X.T@X + optimal_lamda*np.ones(covs))@X.T@X@(npla.inv(X.T@X + optimal_lamda*np.ones(covs))).T
    std_errors = np.sqrt(np.diagonal(cov_matrix))
    t_score = st.t.ppf((1 + 0.95) / 2, X.shape[0] - X.shape[1])
    
    results = [[beta[i] - t_score * std_errors[i], beta[i] + t_score * std_errors[i]] for i in range(6)]
    
    df = pd.DataFrame(results, columns = [0,1])
    df.index = np.array(['Intercept'] + xcolumns)
    
    return df

In [121]:
assignment_4('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', data, folds=5)

,0,1
Intercept,-2.863447,37.884986
Literacy,-0.448337,0.148515
Donations,-0.000674,0.000954
Infants,0.000012,0.001146
Wealth,0.096456,0.516639
Commerce,-0.093880,0.398389
